# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [ ]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [7]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [8]:
import numpy as np

def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00445576  1.4153057  -0.45133692  0.19490376  0.00516991  0.10223453
  0.          0.        ] -> Action: 1
Obs: [-0.00897388  1.4191241  -0.45849857  0.1696736   0.01177964  0.1322072
  0.          0.        ] -> Action: 0
Obs: [-0.0134922   1.422344   -0.45851907  0.14302996  0.0183854   0.13212758
  0.          0.        ] -> Action: 3
Obs: [-0.01793861  1.424966   -0.44947395  0.11647124  0.02317228  0.09574637
  0.          0.        ] -> Action: 0
Obs: [-0.02238503  1.4269884  -0.4494869   0.08980122  0.02795921  0.09574737
  0.          0.        ] -> Action: 3
Obs: [-0.02674389  1.4284194  -0.4384871   0.0635452   0.03053286  0.05147778
  0.          0.        ] -> Action: 0
Obs: [-0.03110275  1.4292504  -0.43849343  0.0368763   0.03310706  0.05148907
  0.          0.        ] -> Action: 2
Obs: [-0.03543043  1.4306611  -0.43563557  0.06263424  0.03595366  0.05693711
  0.          0.        ] -> Action: 0
Obs: [-0.03975802  1.4314718  -0.4356433   0.03595556  0.03879962

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [9]:
from enum import Enum

class Act(Enum):
    LEFT = 1 # động cơ trái
    RIGHT = 3 # động cơ phải
    MAIN = 2 #động cơ chính
    NO_OP = 0 # không làm gì

class Obs(Enum):
    X = 0 #vị trí theo X
    Y = 1 #vị trí theo Y
    VX = 2 # vận tốc theo X
    VY = 3 # vận tốc theo Y
    ANGLE = 4 # Góc nghiêng của tàu
    ANGULAR_VELOCITY = 5 # vận tốc quay
    LEFT_LEG_CONTACT = 6 #chân trái chạm đất chưa
    RIGHT_LEG_CONTACT = 7 # chân phải chạm đất chưa


In [10]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [-0.00393314  1.4135064  -0.39840886  0.11493837  0.0045644   0.0902455
  0.          0.        ] -> Action: 0
Obs: [-0.00786676  1.415515   -0.39787006  0.08925278  0.00902383  0.08919757
  0.          0.        ] -> Action: 0
Obs: [-0.01180058  1.4169244  -0.39788428  0.06260322  0.01348063  0.08914398
  0.          0.        ] -> Action: 0
Obs: [-0.01573458  1.4177339  -0.39789742  0.03593055  0.01793685  0.08913283
  0.          0.        ] -> Action: 0
Obs: [-0.01966858  1.4179436  -0.3979106   0.00926083  0.0223924   0.08911884
  0.          0.        ] -> Action: 0
Obs: [-0.02360277  1.4175537  -0.39792377 -0.01740888  0.02684722  0.08910467
  0.          0.        ] -> Action: 0
Obs: [-0.02753716  1.4165639  -0.39793688 -0.04407865  0.03130135  0.08909073
  0.          0.        ] -> Action: 0
Obs: [-0.03147173  1.4149742  -0.39794993 -0.07074846  0.03575478  0.08907676
  0.          0.        ] -> Action: 0
Obs: [-0.0354063   1.4127848  -0.397963   -0.09741838  0.0402075 

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [11]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [21]:
# Code goes here
def better_reflex_agent(observation):
  """ Viết một agent cải tiến biết điều chỉnh cả trái phải để cân bằng
  khi hạ cánh chứ không phải chỉ sử dụng mỗi động cơ chính và chạy thử 100 lần"""

  # rơi nhanh thì bật động cơ chính
  if observation[Obs.VY.value] <-.3:
    return Act.MAIN.value
  # kiểm tra có nghiêng trái hay quay sang trái không.
  angle = observation[Obs.ANGLE.value]     # góc nghiêng
  angular_velocity = observation[Obs.ANGULAR_VELOCITY.value]  # tốc độ quay
  #bên trái.
  if angle <-.1 or angular_velocity <-.1:
    return Act.RIGHT.value
  #bên phải.
  if angle > .1 or angular_velocity >.1:
    return Act.LEFT.value
  return Act.NO_OP.value

rewards = run_episodes(better_reflex_agent, n=100)
print(f"reward:{rewards}")
print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

reward:[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Average reward: -100.0
Success rate: 0/100
